In [7]:
# Import the PyTorch library
import torch
import torch_directml # This uses pytorch with DirectML (For AMD GPU)


# Print the version of CUDA (Compute Unified Device Architecture), which is a parallel computing platform and application programming interface model created by Nvidia
print(torch.version.cuda)

# Sets dml as the DirectML device
dml = torch_directml.device()
print(dml)

# 
print(torch_directml.device_count())

# Check if a GPU (cuda) is available, otherwise use CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Print the selected device (either 'cuda' or 'cpu')
print(f"selected device: {device}")

block_size = 8
batch_size = 4


11.7
privateuseone:0
1
selected device: cpu


In [8]:
with open('pg420.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Checking file contents
print(len(text))
print(text[:200])
chars = sorted(set(text))
print(chars)
print(len(chars))
vocabulary_size = len(chars)

230469
﻿

Dorothy and the Wizard in Oz


  A Faithful Record of Their Amazing Adventures
    in an Underground World; and How with the
     Aid of Their Friends Zeb Hugson, Eureka
       the Kitten, and Jim 
['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
76


In [9]:
# Tokenizer

string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

print(encode('hello'))
encoded_hello = encode('hello')
decoded_hello = decode(encoded_hello)
print(decoded_hello)

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

[56, 53, 60, 60, 63]
hello
tensor([75,  0,  0, 27, 63, 66, 63, 68, 56, 73,  1, 49, 62, 52,  1, 68, 56, 53,
         1, 46, 57, 74, 49, 66, 52,  1, 57, 62,  1, 38, 74,  0,  0,  0,  1,  1,
        24,  1, 29, 49, 57, 68, 56, 54, 69, 60,  1, 41, 53, 51, 63, 66, 52,  1,
        63, 54,  1, 43, 56, 53, 57, 66,  1, 24, 61, 49, 74, 57, 62, 55,  1, 24,
        52, 70, 53, 62, 68, 69, 66, 53, 67,  0,  1,  1,  1,  1, 57, 62,  1, 49,
        62,  1, 44, 62, 52, 53, 66, 55, 66, 63])


In [10]:
# Select data to Train/Validate

# Select 80% of the data to train
n = int(0.8*len(data))
train_data = data[:n]
# Validate the model with the remaining 20%
val_data = data[n:]


In [11]:
# Train the model

# Break the data in blocks
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([75]) target is tensor(0)
when input is tensor([75,  0]) target is tensor(0)
when input is tensor([75,  0,  0]) target is tensor(27)
when input is tensor([75,  0,  0, 27]) target is tensor(63)
when input is tensor([75,  0,  0, 27, 63]) target is tensor(66)
when input is tensor([75,  0,  0, 27, 63, 66]) target is tensor(63)
when input is tensor([75,  0,  0, 27, 63, 66, 63]) target is tensor(68)
when input is tensor([75,  0,  0, 27, 63, 66, 63, 68]) target is tensor(56)
